In [1]:
%load_ext autoreload
%autoreload 2
import sentenceSimilarityMethods

In [2]:
import json
import os
import nltk
import re
import pandas as pd
import JobOffers
import tensorflow_hub as hub
import numpy as np
import tensorflow as tf

In [3]:
from scipy import spatial

In [4]:
autoreload JobOffers

In [5]:
alljobs = JobOffers.readJsonFile("output_V1.1.json")

In [6]:
import tokenizers
from easynmt import EasyNMT
model = EasyNMT('opus-mt')

In [7]:
#alljobs = JobOffers.addTranslation(alljobs, "output_V1.1.translate.json", maxWrite=100)

In [8]:
alljobs = JobOffers.readJsonFile("output_V1.1.translate.json")

In [9]:
alljobs

[{'ID': 12220025,
  'ISCO': '21420',
  'SKILLS': 'Feingefühl;Bauingenieurwesen',
  'BRANCHE': 'Architektur und Planung',
  'ARBEITSORT': 'CH-3778 Schönried',
  'KANTONE': 'BE',
  'FIRMENGROESSE': 'Kleinstunternehmen',
  'BERUFSERFAHRUNG': 'Bauleitung;Erschliessung von Grundstücken; Bauträger;Projektingenieur;Projektierung;Bauprojektleiter',
  'AUSBILDUNGSNIVEAU': 'BACHELOR;MASTER;COLLEGE;DIPLOMA;UAS',
  'ZUDATUM': '2016-07-14',
  'ABDATUM': '2018-07-03',
  'FIRMA': 'Haldi Design AG',
  'JOBTITEL': 'Bauingenieur (m/w)',
  'JOBS': 'Bauingenieur',
  'CONTENT': '<p>Wir sind ein erfolgreiches und unabhängiges Ingenieurbüro in Schönried.<br><br>Zur Verstärkung unseres Teams und um die vielfältigen, interessanten Objekte optimal bearbeiten zu können, suchen wir</p>\n<h2>Bauingenieur (m/w)</h2>\n<h3>Ihre Arbeitsgebiete</h3>\n<p>Konstruktiver Ingenieurbau sowohl in Projektierung&nbsp; als auch in der Bauleitung allgemeiner Tief- und Strassenbau.</p>\n<h3>Ihr Profil</h3>\n<p>Sie verfügen über ei

In [10]:
jobsDf = pd.DataFrame(np.array(([x["ID"] for x in alljobs], [x["ISCO"] for x in alljobs], [x["TOKENIZED_JOBS"] for x in alljobs] ), dtype='object').T, columns = ["JOB_ID", "ISCO", "JOB_CONTENT"])




In [232]:
matchid = 12720277
for j in alljobs:
    if j["ID"] == matchid:
        print(j["CONTENT"])

Places libres à consulter sur lien URL ci-dessousLandwirt/in EFZSitten (VS (fr))

Sprachen 

fr


Anzahl Lehrstellen 

2


Swissdoc-Nummer 

0.130.12.0


Aktualisiert16.05.2017


In [11]:
jobsSentences = np.array([y for x in jobsDf["JOB_CONTENT"] for y in x])

In [12]:
#Activities Json
activitesJson = JobOffers.readJsonFile("Dataset_Classes.json", encoding="utf-8")

In [13]:
#Old activities CSV that I will work with for now
oldActs = pd.read_csv(os.path.expanduser("~/Desktop/Thesis/Data/Activities.csv"), error_bad_lines=False)

In [14]:
oldActsDf = oldActs.rename(columns={'O*Net\nMétiers verts' : "ACTIVITY_ID", 'O"Net: Détails des activités pour les métiers verts' : "ACTIVITY_CONTENT"})

In [15]:
slimmedActsDf = oldActsDf[["ACTIVITY_ID", "ACTIVITY_CONTENT"]][~oldActsDf["ACTIVITY_CONTENT"].isna()]

In [16]:
slimmedActsDf.loc[slimmedActsDf["ACTIVITY_ID"].isna(), 'ACTIVITY_ID'] = slimmedActsDf[slimmedActsDf["ACTIVITY_ID"].isna()].index

In [17]:
slimmedActsDf

,ACTIVITY_ID,ACTIVITY_CONTENT
0,0,Where and how are the information and data gai...
1,1,"Observing, receiving, and otherwise obtaining..."
3,4.A.1.a.1.G01,Collect geographic information systems (GIS) d...
4,4.A.1.a.1.G02,Collect geological data.
5,4.A.1.a.1.G03,Collect data about green practices or programs.
...,...,...
225,4.A.4.b.6.G08,Provide technical guidance on green projects.
226,226,Performing day-to-day administrative tasks suc...
227,4.A.4.c.1.G01,Obtain permits.
228,228,"Recruiting, interviewing, selecting, hiring, ..."


In [18]:
activitiesSentences = slimmedActsDf["ACTIVITY_CONTENT"].to_numpy()

In [19]:
allSentences = np.concatenate((jobsSentences, activitiesSentences))

In [20]:
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [21]:
#module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
#@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
#model = hub.load(module_url)
#def embed(input):
  #return model(input)

In [22]:
model = hub.load("/Users/emilysharata/Desktop/Thesis/universal-sentence-encoder_4/")
def embed(input):
  return model(input)

In [23]:
len(slimmedActsDf)

219

In [24]:
allSentences

array(['We are a successful and independent engineering office in Schönried',
       ' To strengthen our team and to optimally process the manifold interesting objects, we are looking for civil engineering MW of their work areas of constructive engineering construction both in project planning and in the construction management of general low and road construction',
       ' Your profile You have a completed studies as a civil engineer ETHTUFH',
       ..., 'Obtain permits.',
       ' Recruiting, interviewing, selecting, hiring, and promoting employees in an organization.',
       ' Monitoring and controlling resources and overseeing the spending of money.'],
      dtype=object)

In [25]:
for i, x in enumerate(allSentences):
    if type(x) != str:
        print(i, x)

In [26]:
allSentences = np.concatenate((jobsSentences, activitiesSentences))

In [27]:
allEmbeddings = embed(allSentences)

In [28]:
len(allEmbeddings)

1044

In [29]:
allEmbeddings

<tf.Tensor: shape=(1044, 512), dtype=float32, numpy=
array([[-0.04822776, -0.02858711, -0.06559009, ...,  0.01871002,
        -0.0613087 , -0.00590269],
       [ 0.0207816 ,  0.03388791,  0.04297301, ...,  0.0401625 ,
        -0.04559466,  0.00078988],
       [ 0.04384393, -0.05652945,  0.00356796, ...,  0.00565387,
        -0.02749583, -0.04372038],
       ...,
       [ 0.01992017, -0.07031337,  0.01158184, ...,  0.02191224,
         0.03360986, -0.01287735],
       [ 0.0046779 , -0.07686147, -0.00192719, ...,  0.04384581,
        -0.01931715,  0.04942223],
       [ 0.06814821, -0.04083548,  0.06860898, ...,  0.02696883,
        -0.0426277 ,  0.07055186]], dtype=float32)>

In [30]:
jobBoundaries = np.cumsum([len(x) for x in jobsDf["JOB_CONTENT"]])
jobBoundaries

array([ 11,  21,  28,  30,  51,  53,  69,  70,  83,  96, 108, 110, 111,
       119, 121, 123, 135, 139, 159, 179, 188, 199, 201, 203, 210, 219,
       227, 236, 238, 255, 272, 274, 285, 307, 313, 327, 334, 338, 352,
       353, 364, 369, 376, 396, 404, 414, 428, 462, 469, 499, 501, 506,
       517, 524, 531, 534, 547, 566, 573, 582, 593, 595, 597, 599, 601,
       608, 617, 641, 661, 674, 676, 690, 692, 694, 696, 704, 706, 727,
       729, 739, 741, 743, 745, 747, 749, 760, 762, 775, 777, 784, 792,
       794, 796, 798, 800, 802, 812, 821, 823, 825])

In [31]:
def matchToJob(row):
    first = jobBoundaries[row.name-1] if row.name != 0 else 0
    last = jobBoundaries[row.name]
    return allEmbeddings[first:last]

In [32]:
jobsDf["JOB_SCORES"] = jobsDf.apply(matchToJob, axis=1)

In [33]:
jobsDf

,JOB_ID,ISCO,JOB_CONTENT,JOB_SCORES
0,12220025,21420,[We are a successful and independent engineeri...,"((tf.Tensor(-0.04822776, shape=(), dtype=float..."
1,10814567,21420,[We are a prestigious and successful engineeri...,"((tf.Tensor(-0.020631889, shape=(), dtype=floa..."
2,12401674,21420,[Bill Weyermann Partner AG has been producing ...,"((tf.Tensor(0.055472188, shape=(), dtype=float..."
3,12473085,62100,[Administration Communale de Valderuzforstarti...,"((tf.Tensor(0.019481275, shape=(), dtype=float..."
4,12905109,26220;24210,[The Straumann Group is experiencing internati...,"((tf.Tensor(-0.022287764, shape=(), dtype=floa..."
...,...,...,...,...
95,13469747,62100,[Commune de Lausanne Service of the Parcs Et D...,"((tf.Tensor(0.06673486, shape=(), dtype=float3..."
96,13494906,24210,[Quality Assurance Engineer Bern Switzerland Y...,"((tf.Tensor(-0.010205403, shape=(), dtype=floa..."
97,13517245,21420;31180,[Solution-oriented creative and dynamic We are...,"((tf.Tensor(-0.055482067, shape=(), dtype=floa..."
98,13461980,21620,[Green Blue Gardener Garden and Landscaping Ef...,"((tf.Tensor(-0.008430735, shape=(), dtype=floa..."


In [34]:
slimmedActsDf["ACTIVITY_SCORES"] = allEmbeddings[-1*(len(slimmedActsDf)):]

In [35]:
len(slimmedActsDf["ACTIVITY_SCORES"][0])

512

In [36]:
dist = spatial.distance.cosine(jobsDf["JOB_SCORES"][0][0], slimmedActsDf["ACTIVITY_SCORES"][0])

In [37]:
dist

0.9204563945531845

In [38]:
resultsDf = pd.DataFrame()

In [39]:
resultsDf["JOB_ID"] = jobsDf["JOB_ID"]

In [40]:
resultsDf["JOB_SCORES"] = jobsDf["JOB_SCORES"]

In [41]:
resultsDf = resultsDf.merge(slimmedActsDf, how = "cross")

In [42]:
len(resultsDf["ACTIVITY_SCORES"])

21900

In [43]:
resultsDf = resultsDf[["JOB_ID", "ACTIVITY_ID", "JOB_SCORES", "ACTIVITY_SCORES"]]


In [44]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES
0,12220025,0,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3..."
1,12220025,1,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32..."
2,12220025,4.A.1.a.1.G01,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.061349783, shape=(), dtype=float3..."
3,12220025,4.A.1.a.1.G02,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.04309698, shape=(), dtype=float32..."
4,12220025,4.A.1.a.1.G03,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.035856005, shape=(), dtype=float3..."
...,...,...,...,...
21895,13462067,4.A.4.b.6.G08,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0028080945, shape=(), dtype=float..."
21896,13462067,226,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(-0.008161316, shape=(), dtype=float..."
21897,13462067,4.A.4.c.1.G01,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.019920168, shape=(), dtype=float3..."
21898,13462067,228,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0046779006, shape=(), dtype=float..."


In [45]:
row = resultsDf.loc[0,:]

In [46]:
def compareSentences(row):
    for sentence in resultsDf["JOB_SCORES"]:
        return spatial.distance.cosine(row["JOB_SCORES"][sentence], row["ACTIVITY_SCORES"])

In [47]:
def distCalc(row):
    distArray = []
    for entry in row["JOB_SCORES"] :
        dist = spatial.distance.cosine(entry, row["ACTIVITY_SCORES"])
        distArray.append(dist)
    return distArray
    

In [48]:
row = resultsDf.loc[0,:]

In [49]:
jobsDfRow = jobsDf.loc[0,:]

In [50]:
resultsDf["DISTANCES"] = resultsDf.apply(distCalc, axis =1)

In [51]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES,DISTANCES
0,12220025,0,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3...","[0.9204563945531845, 0.9612990096211433, 0.896..."
1,12220025,1,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32...","[1.1060239672660828, 0.9814335498958826, 0.960..."
2,12220025,4.A.1.a.1.G01,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.061349783, shape=(), dtype=float3...","[1.02582386136055, 1.048024758696556, 0.852094..."
3,12220025,4.A.1.a.1.G02,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.04309698, shape=(), dtype=float32...","[1.0590376444160938, 1.0112552838400006, 0.829..."
4,12220025,4.A.1.a.1.G03,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.035856005, shape=(), dtype=float3...","[1.0500608645379543, 1.095018446445465, 0.9780..."
...,...,...,...,...,...
21895,13462067,4.A.4.b.6.G08,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0028080945, shape=(), dtype=float...","[0.9995186589076184, 1.2212068736553192]"
21896,13462067,226,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(-0.008161316, shape=(), dtype=float...","[0.977263942360878, 0.9934241622686386]"
21897,13462067,4.A.4.c.1.G01,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.019920168, shape=(), dtype=float3...","[0.9572820700705051, 0.9627352058887482]"
21898,13462067,228,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0046779006, shape=(), dtype=float...","[0.8675366938114166, 0.8801782131195068]"


In [271]:
def minDistanceFinder(row) : 
    minDistance = min(row["DISTANCES"])
    return minDistance

In [318]:
def minDistanceSentence(row, jobs) : 
    minDistanceIdx = np.argmin(row["DISTANCES"])
    df = jobs[jobs["JOB_ID"] == row["JOB_ID"]]
    if df.empty:
        return ""
    content = df.iloc[0]["JOB_CONTENT"]
    return content[minDistanceIdx]

In [319]:
minDistanceSentence(resultsDf.iloc[6,:], jobsDf)

' You will get the opportunity to grow on the task'

In [320]:
df = jobsDf[jobsDf["JOB_ID"] == 13462067]
df.iloc[0]["JOB_CONTENT"]

['Haas Gartenbau Aggartenner in Garden and landscaping Evzbern Be Languages Number of apprenticeships 1 Swissdoc number 0',
 ' 9 Updated19']

In [321]:
resultsDf["LOWEST_DISTANCE"] = resultsDf.apply(minDistanceFinder, axis =1)
resultsDf["LOWEST_DISTANCE_SENTENCE"] = resultsDf.apply(minDistanceSentence, args=[jobsDf], axis =1)


SyntaxError: unexpected EOF while parsing (<ipython-input-322-513313685a93>, line 1)

In [323]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES,DISTANCES,LOWEST_DISTANCE,AVERAGE_DISTANCE_3SENT,LOWEST_DISTANCE_SENTENCE
0,12220025,0,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3...","[0.9204563945531845, 0.9612990096211433, 0.896...",0.670784,0.764877,You want to gain experience in configuration ...
1,12220025,1,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32...","[1.1060239672660828, 0.9814335498958826, 0.960...",0.730214,0.841840,Further information and application We are at...
2,12220025,4.A.1.a.1.G01,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.061349783, shape=(), dtype=float3...","[1.02582386136055, 1.048024758696556, 0.852094...",0.852094,0.910633,Your profile You have a completed studies as ...
3,12220025,4.A.1.a.1.G02,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.04309698, shape=(), dtype=float32...","[1.0590376444160938, 1.0112552838400006, 0.829...",0.814106,0.850310,Haldi Design AG Schönried
4,12220025,4.A.1.a.1.G03,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.035856005, shape=(), dtype=float3...","[1.0500608645379543, 1.095018446445465, 0.9780...",0.923093,0.944726,Haldi Design AG Schönried
...,...,...,...,...,...,...,...,...
21895,13462067,4.A.4.b.6.G08,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0028080945, shape=(), dtype=float...","[0.9995186589076184, 1.2212068736553192]",0.999519,1.110363,Haas Gartenbau Aggartenner in Garden and lands...
21896,13462067,226,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(-0.008161316, shape=(), dtype=float...","[0.977263942360878, 0.9934241622686386]",0.977264,0.985344,Haas Gartenbau Aggartenner in Garden and lands...
21897,13462067,4.A.4.c.1.G01,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.019920168, shape=(), dtype=float3...","[0.9572820700705051, 0.9627352058887482]",0.957282,0.960009,Haas Gartenbau Aggartenner in Garden and lands...
21898,13462067,228,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0046779006, shape=(), dtype=float...","[0.8675366938114166, 0.8801782131195068]",0.867537,0.873857,Haas Gartenbau Aggartenner in Garden and lands...


In [55]:
#resultsDf.loc[14851,:]

In [56]:
#resultsDf["TOP_ACTIVITY"] = resultsDf.apply(topActivityFinder, axis = 1)

In [57]:
#for id in jobsDf["JOB_ID"] : 
   # print (id)
   # break 

In [324]:
for id in jobsDf["JOB_ID"] :
    job1 = resultsDf[resultsDf["JOB_ID"] == id]
    minEntry = job1["LOWEST_DISTANCE"].idxmin()
    rowMatch = job1.loc[minEntry,:]
    content = slimmedActsDf[slimmedActsDf["ACTIVITY_ID"] == rowMatch["ACTIVITY_ID"]]["ACTIVITY_CONTENT"]
    print(id, minEntry, rowMatch["ACTIVITY_ID"], content)

12220025 194 4.A.4.b.1.G09 205    Manage engineering or construction activities.
Name: ACTIVITY_CONTENT, dtype: object
10814567 413 4.A.4.b.1.G09 205    Manage engineering or construction activities.
Name: ACTIVITY_CONTENT, dtype: object
12401674 632 4.A.4.b.1.G09 205    Manage engineering or construction activities.
Name: ACTIVITY_CONTENT, dtype: object
12473085 853 207 207     Identifying the educational needs of others, ...
Name: ACTIVITY_CONTENT, dtype: object
12905109 1061 196 196     Getting members of a group to work together t...
Name: ACTIVITY_CONTENT, dtype: object
12523090 1173 4.A.2.b.1.G08 82    Select materials for use in green applications...
Name: ACTIVITY_CONTENT, dtype: object
11449790 1471 4.A.3.b.6.G04 168    Prepare environmental reports or documentation.
Name: ACTIVITY_CONTENT, dtype: object
12848898 1744 4.A.4.b.6.G05 222    Advise others on green engineering or design i...
Name: ACTIVITY_CONTENT, dtype: object
12849192 1772 4.A.1.b.2.G03 22    Inspect heating, v

13466616 19709 229 229     Monitoring and controlling resources and over...
Name: ACTIVITY_CONTENT, dtype: object
13485931 19748 4.A.2.a.2.G04 41    Compute expenses or savings associated with gr...
Name: ACTIVITY_CONTENT, dtype: object
13463812 20015 4.A.2.b.2.G03 91    Design green agricultural technologies.
Name: ACTIVITY_CONTENT, dtype: object
13463898 20234 4.A.2.b.2.G03 91    Design green agricultural technologies.
Name: ACTIVITY_CONTENT, dtype: object
13461966 20371 4.A.1.a.1.G03 5    Collect data about green practices or programs.
Name: ACTIVITY_CONTENT, dtype: object
13461744 20782 207 207     Identifying the educational needs of others, ...
Name: ACTIVITY_CONTENT, dtype: object
13469747 21010 216 216    Advise others about land management or conserv...
Name: ACTIVITY_CONTENT, dtype: object
13494906 21183 4.A.3.b.6.G06 170    Prepare work requests or proposals.
Name: ACTIVITY_CONTENT, dtype: object
13517245 21454 4.A.4.b.6.G05 222    Advise others on green engineering or desig

In [325]:
def bestMatchReturn(row, results, actsDf, match, column_name) :
    bestMatch = results[results["JOB_ID"] == row["JOB_ID"]]
    minEntry = bestMatch[match].idxmin()
    rowMatch = bestMatch.loc[minEntry,:]     
    content = actsDf[actsDf["ACTIVITY_ID"] == rowMatch["ACTIVITY_ID"]]                  
    return content[column_name].iloc[0]            


In [326]:
def averageNSentences(row, n):
    return np.average(np.sort(row["DISTANCES"])[:n])

In [327]:
resultsDf["AVERAGE_DISTANCE_3SENT"] = resultsDf.apply(averageNSentences, args=[3], axis=1)

In [328]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES,DISTANCES,LOWEST_DISTANCE,AVERAGE_DISTANCE_3SENT,LOWEST_DISTANCE_SENTENCE
0,12220025,0,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3...","[0.9204563945531845, 0.9612990096211433, 0.896...",0.670784,0.764877,You want to gain experience in configuration ...
1,12220025,1,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32...","[1.1060239672660828, 0.9814335498958826, 0.960...",0.730214,0.841840,Further information and application We are at...
2,12220025,4.A.1.a.1.G01,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.061349783, shape=(), dtype=float3...","[1.02582386136055, 1.048024758696556, 0.852094...",0.852094,0.910633,Your profile You have a completed studies as ...
3,12220025,4.A.1.a.1.G02,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.04309698, shape=(), dtype=float32...","[1.0590376444160938, 1.0112552838400006, 0.829...",0.814106,0.850310,Haldi Design AG Schönried
4,12220025,4.A.1.a.1.G03,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.035856005, shape=(), dtype=float3...","[1.0500608645379543, 1.095018446445465, 0.9780...",0.923093,0.944726,Haldi Design AG Schönried
...,...,...,...,...,...,...,...,...
21895,13462067,4.A.4.b.6.G08,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0028080945, shape=(), dtype=float...","[0.9995186589076184, 1.2212068736553192]",0.999519,1.110363,Haas Gartenbau Aggartenner in Garden and lands...
21896,13462067,226,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(-0.008161316, shape=(), dtype=float...","[0.977263942360878, 0.9934241622686386]",0.977264,0.985344,Haas Gartenbau Aggartenner in Garden and lands...
21897,13462067,4.A.4.c.1.G01,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.019920168, shape=(), dtype=float3...","[0.9572820700705051, 0.9627352058887482]",0.957282,0.960009,Haas Gartenbau Aggartenner in Garden and lands...
21898,13462067,228,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0046779006, shape=(), dtype=float...","[0.8675366938114166, 0.8801782131195068]",0.867537,0.873857,Haas Gartenbau Aggartenner in Garden and lands...


In [60]:
#def activityMatcher(row) :
 #   for id in jobsDf["JOB_ID"] :
  #      bestOverallActivityMatch = np.argmin(resultsDf[row["JOB_ID"] == id]["LOWEST_DISTANCE"])
  #  return bestOverallActivityMatch 
    

In [329]:
column_names = ["JOB_ID", "ISCO", "JOB_CONTENT", "LOWEST_DISTANCE", "BEST_ACTIVITY_MATCH", "BEST_ACTIVITY_ID", "TOP_ACTIVITY_MATCHES", "TOP ACTIVITY_IDS"]

In [330]:
topResultsDf = pd.DataFrame(columns = column_names)

In [331]:
topResultsDf["JOB_ID"] = jobsDf["JOB_ID"]

In [332]:
topResultsDf["ISCO"] = jobsDf["ISCO"]

In [333]:
topResultsDf["JOB_CONTENT"] = jobsDf.apply(lambda x: " ".join(x["JOB_CONTENT"]), axis=1)

In [192]:
topResultsDf["LOWEST_DISTANCE"] = resultsDf["LOWEST_DISTANCE"]
topResultsDf["AVERAGE_DISTANCE_3SENT"] = resultsDf["AVERAGE_DISTANCE_3SENT"]

In [334]:
topResultsDf

,JOB_ID,ISCO,JOB_CONTENT,LOWEST_DISTANCE,BEST_ACTIVITY_MATCH,BEST_ACTIVITY_ID,TOP_ACTIVITY_MATCHES,TOP ACTIVITY_IDS
0,12220025,21420,We are a successful and independent engineerin...,NaN,NaN,NaN,NaN,NaN
1,10814567,21420,We are a prestigious and successful engineerin...,NaN,NaN,NaN,NaN,NaN
2,12401674,21420,Bill Weyermann Partner AG has been producing e...,NaN,NaN,NaN,NaN,NaN
3,12473085,62100,Administration Communale de Valderuzforstartin...,NaN,NaN,NaN,NaN,NaN
4,12905109,26220;24210,The Straumann Group is experiencing internatio...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
95,13469747,62100,Commune de Lausanne Service of the Parcs Et Do...,NaN,NaN,NaN,NaN,NaN
96,13494906,24210,Quality Assurance Engineer Bern Switzerland Yo...,NaN,NaN,NaN,NaN,NaN
97,13517245,21420;31180,Solution-oriented creative and dynamic We are ...,NaN,NaN,NaN,NaN,NaN
98,13461980,21620,Green Blue Gardener Garden and Landscaping Efl...,NaN,NaN,NaN,NaN,NaN


In [335]:
val = bestMatchReturn(row, resultsDf, slimmedActsDf, "ACTIVITY_CONTENT")

TypeError: bestMatchReturn() missing 1 required positional argument: 'column_name'

In [336]:
topResultsDf["BEST_ACTIVITY_MATCH"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "LOWEST_DISTANCE", "ACTIVITY_CONTENT"), axis=1)
topResultsDf["BEST_ACTIVITY_MATCH_3SENT"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "AVERAGE_DISTANCE_3SENT", "ACTIVITY_CONTENT"), axis=1)

In [337]:
topResultsDf["BEST_ACTIVITY_ID"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "LOWEST_DISTANCE", "ACTIVITY_ID"), axis=1)
topResultsDf["BEST_ACTIVITY_ID_3SENT"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "AVERAGE_DISTANCE_3SENT", "ACTIVITY_ID"), axis=1)
topResultsDf

,JOB_ID,ISCO,JOB_CONTENT,LOWEST_DISTANCE,BEST_ACTIVITY_MATCH,BEST_ACTIVITY_ID,TOP_ACTIVITY_MATCHES,TOP ACTIVITY_IDS,BEST_ACTIVITY_MATCH_3SENT,BEST_ACTIVITY_ID_3SENT
0,12220025,21420,We are a successful and independent engineerin...,NaN,Manage engineering or construction activities.,4.A.4.b.1.G09,NaN,NaN,Manage engineering or construction activities.,4.A.4.b.1.G09
1,10814567,21420,We are a prestigious and successful engineerin...,NaN,Manage engineering or construction activities.,4.A.4.b.1.G09,NaN,NaN,Manage engineering or construction activities.,4.A.4.b.1.G09
2,12401674,21420,Bill Weyermann Partner AG has been producing e...,NaN,Manage engineering or construction activities.,4.A.4.b.1.G09,NaN,NaN,Manage engineering or construction activities.,4.A.4.b.1.G09
3,12473085,62100,Administration Communale de Valderuzforstartin...,NaN,"Identifying the educational needs of others, ...",207,NaN,NaN,"Identifying the educational needs of others, ...",207
4,12905109,26220;24210,The Straumann Group is experiencing internatio...,NaN,Getting members of a group to work together t...,196,NaN,NaN,Developing specific goals and plans to priorit...,117
...,...,...,...,...,...,...,...,...,...,...
95,13469747,62100,Commune de Lausanne Service of the Parcs Et Do...,NaN,Advise others about land management or conserv...,216,NaN,NaN,"Identifying the educational needs of others, ...",207
96,13494906,24210,Quality Assurance Engineer Bern Switzerland Yo...,NaN,Prepare work requests or proposals.,4.A.3.b.6.G06,NaN,NaN,Evaluate materials to assess quality or ensure...,4.A.2.a.3.G01
97,13517245,21420;31180,Solution-oriented creative and dynamic We are ...,NaN,Advise others on green engineering or design i...,4.A.4.b.6.G05,NaN,NaN,Advise others on green engineering or design i...,4.A.4.b.6.G05
98,13461980,21620,Green Blue Gardener Garden and Landscaping Efl...,NaN,Design green agricultural technologies.,4.A.2.b.2.G03,NaN,NaN,Operate robotic or automated equipment.,4.A.3.a.3.G04


In [229]:
print(topResultsDf.iloc[4]["JOB_CONTENT"])
print(topResultsDf.iloc[4]["BEST_ACTIVITY_MATCH"])
print(topResultsDf.iloc[4]["BEST_ACTIVITY_MATCH_3SENT"])

The Straumann Group is experiencing international success in the field of implantology and dental reconstruction as well as tissue regeneration  For more than 60 years we have been focusing our skills on the research and development of optimal solutions for our customers and patients  Implantable medical devices produced on our Villeret site are certified according to MDD 9342ECC ISO 13485 ISO 14001 and FDACGMP 21 CFR 820  In order to ensure that all valid quality documents for the Villeret site are correctly defined in the relevant departments they are Update and made available to users we complete our team with a quality documentation specialist  Your function as part of the management of our Quality System You are part of a team responsible for document management process management and training  In this function you evaluate and process requests for changes attend the departments responsible for the definition and development of the various quality documents and propose continuous 

In [230]:
topResultsDf.to_csv("PrelimResults.csv")